In [ ]:
%%bash
# 安裝 llama.cpp
git clone https://github.com/ggerganov/llama.cpp.git
cd llama.cpp
LLAMA_CUBLAS=1 make
#pip install -r requirements.txt

In [3]:
#  IProgress ipywidgets
!pip install  IProgress ipywidgets -q

In [4]:
# Library及TOKEN
from huggingface_hub import create_repo, HfApi, ModelCard
hf_token="hf_"  # 修改為你的hf_token
api = HfApi()
hf_api = HfApi(
    endpoint="https://huggingface.co", # Can be a Private Hub endpoint.
    token=hf_token, # Token is not persisted on the machine.
)

In [ ]:
# 從本地端轉檔
# Quantization methods: `iqs_2`, `iqs_3`, `q2_k`, `q3_k_l`, `q3_k_m`, `q3_k_s`, `q4_0`, `q4_1`, `q4_k_m`, `q4_k_s`, `q5_0`, `q5_1`, `q5_k_m`, `q5_k_s`, `q6_k`, `q8_0`, etc.
QUANTIZATION_FORMAT = "q5_k_m" #  修改為你的量化格式, 可以多種
QUANTIZATION_METHODS = QUANTIZATION_FORMAT.replace(" ", "").split(",")

# MODEL ID
USERNAME = "c00cjz00" # # 修改為你的HF帳號 
MODEL_ID="meta-llama/Meta-Llama-3-8B-Instruct" # 修改為你參考來源下載 範例model card
MODEL_NAME = "Meta-Llama-3-8B-Instruct"  # 修改為你的新命名model name 
MODEL_SOURCE = "~/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B-Instruct/snapshots/c4a54320a52ed5f88b7a2f84496903ea4ff07b45"   # 修改為你的本地端要轉檔的model

# Convert to fp16
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp16.bin"
!mkdir -p {MODEL_NAME}
!./llama.cpp/convert-hf-to-gguf.py {MODEL_SOURCE} --outtype f16 --outfile {fp16}

# Quantize the model for each method in the QUANTIZATION_METHODS list
for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{method.upper()}.gguf"
    !./llama.cpp/quantize {fp16} {qtype} {method}

In [ ]:
# Create model card 並上傳
card = ModelCard.load(MODEL_ID)
card.data.tags.append("autoquant")
card.data.tags.append("gguf")
card.save(f'{MODEL_NAME}/README.md')

# Upload model
create_repo(
    repo_id = f"{USERNAME}/{MODEL_NAME}-GGUF",
    repo_type="model",
    exist_ok=True,
    token=hf_token
)


api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{USERNAME}/{MODEL_NAME}-GGUF",
    allow_patterns=["*.gguf","$.md"],
    token=hf_token
)

meta-llama-3-8b-instruct.Q5_K_M.gguf:   2%|▏         | 93.8M/5.73G [00:09<10:20, 9.09MB/s]  